In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as si
import time

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import models, layers, activations
from tensorflow.keras.layers import BatchNormalization

np.set_printoptions(precision = 3, suppress = True)

In [2]:
ES_EZ = pd.read_csv('/Users/gaojinglun/Desktop/RSG/ES_EZ.csv')
ES_EZ = ES_EZ.drop(['Unnamed: 0'], axis = 1)
df = ES_EZ[['Put.Call', 'Strike.Price', 'Settlement', 'Open.Interest', 
            'Delta', 'Implied.Volatility', 'futures.price', 'Time.to.maturity', 'Risk.Free.Rate']]
df = df.dropna()
df.head()

,Put.Call,Strike.Price,Settlement,Open.Interest,Delta,Implied.Volatility,futures.price,Time.to.maturity,Risk.Free.Rate
0,C,2200,1059.0,826.0,0.99787,0.338267,3259.0,0.213699,0.0154
1,C,2250,1009.1,728.0,0.99661,0.326829,3259.0,0.213699,0.0154
2,C,2270,989.1,1.0,0.99586,0.323456,3259.0,0.213699,0.0154
3,C,2300,959.2,212.0,0.99474,0.317834,3259.0,0.213699,0.0154
4,C,2310,949.2,42.0,0.99466,0.314282,3259.0,0.213699,0.0154


In [3]:
ES_EZ_call = df[df['Put.Call'] == 'C']
ES_EZ_call_X = ES_EZ_call[['futures.price', 'Strike.Price', 'Time.to.maturity', 
                           'Risk.Free.Rate', 'Implied.Volatility']]
ES_EZ_call_y = ES_EZ_call['Settlement']

ES_EZ_put = df[df['Put.Call'] == 'P']
ES_EZ_put_X = ES_EZ_put[['futures.price', 'Strike.Price', 'Time.to.maturity', 
                           'Risk.Free.Rate', 'Implied.Volatility']]
ES_EZ_put_y = ES_EZ_put['Settlement']
ES_EZ_call.shape, ES_EZ_put.shape

((13705, 9), (16263, 9))

In [4]:
X_train, X_test, y_train, y_test = train_test_split(ES_EZ_call_X, ES_EZ_call_y, test_size = 0.2, random_state = 123)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((10964, 5), (2741, 5), (10964,), (2741,))

In [5]:
# Standardize the features
scalerX = MinMaxScaler().fit(X_train)
X_train = pd.DataFrame(scalerX.transform(X_train), columns = ES_EZ_call_X.columns.values) 
X_test = pd.DataFrame(scalerX.transform(X_test), columns = ES_EZ_call_X.columns.values) 

scalery = MinMaxScaler().fit(y_train.values.reshape(-1, 1))
y_train = scalery.transform(y_train.values.reshape(-1, 1))
y_test = scalery.transform(y_test.values.reshape(-1, 1))

In [6]:
np.random.seed(123)
tf.random.set_seed(123)

In [7]:
def plot_loss(history):
    plt.figure(figsize = (8,6))
    plt.plot(history.history['loss'], label = 'loss')
    plt.plot(history.history['val_loss'], label = 'val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('loss')
    plt.legend()
    plt.grid(True)
    
def plot_time(time):
    print('The total training time is {} seconds.'.format(np.round(np.sum(time), 1)))
    plt.figure(figsize = (8,6))
    plt.plot(np.arange(1, len(time) + 1), time)
    plt.xlabel('Epoch')
    plt.ylabel('Time for each epoch')
    plt.grid(True)

In [8]:
class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs = {}):
        self.times = []

    def on_epoch_begin(self, batch, logs = {}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs = {}):
        self.times.append(time.time() - self.epoch_time_start)

In [9]:
learning_rate = [1e-2, 1e-3, 1e-4]
History = []
times = []
earlyStop = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 3)
for lr in learning_rate:
    model = models.Sequential()
    # Input layer
    model.add(layers.Dense(units = 5))
    
    numLayers = 4
    for i in range(numLayers):
        model.add(layers.Dense(units = 100))
        model.add(BatchNormalization())
        model.add(layers.Activation(activations.elu))
        model.add(layers.Dropout(0.25))

    # Output layer
    model.add(layers.Dense(units = 1, activation = "exponential"))

    model.compile(
        optimizer = keras.optimizers.Adam(learning_rate = lr), loss = "mean_squared_error"
    )
    
    time_callback = TimeHistory()

    history = model.fit(X_train, 
        y_train, 
        epochs = 100, 
        verbose = 0,
        batch_size = 32, 
        callbacks = [earlyStop, time_callback],
        validation_split = 0.2)
    BSNN_train_pred = model.predict(X_train)
    BSNN_test_pred = model.predict(X_test)
    print('When the learning rate is', lr)
    print("The training R squared is", 
          np.round(r2_score(y_train, BSNN_train_pred), 4))
    print("The testing R squared is", 
          np.round(r2_score(y_test, BSNN_test_pred), 4))
    print('-------------------------------------------------------------------')
    History.append(history)
    times.append(time_callback.times)
    path = "/Users/gaojinglun/Desktop/RSG/ANN_call_TrainOnESEZ_withLR" + str(lr)
    model.save(path)

When the learning rate is 0.01
The training R squared is 0.9805
The testing R squared is 0.9809
-------------------------------------------------------------------
INFO:tensorflow:Assets written to: /Users/gaojinglun/Desktop/RSG/ANN_call_TrainOnESEZ_withLR0.01/assets
When the learning rate is 0.001
The training R squared is 0.9604
The testing R squared is 0.9611
-------------------------------------------------------------------
INFO:tensorflow:Assets written to: /Users/gaojinglun/Desktop/RSG/ANN_call_TrainOnESEZ_withLR0.001/assets
When the learning rate is 0.0001
The training R squared is 0.9706
The testing R squared is 0.9709
-------------------------------------------------------------------
INFO:tensorflow:Assets written to: /Users/gaojinglun/Desktop/RSG/ANN_call_TrainOnESEZ_withLR0.0001/assets


In [10]:
X_train, X_test, y_train, y_test = train_test_split(ES_EZ_put_X, ES_EZ_put_y, test_size = 0.2, random_state = 123)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((13010, 5), (3253, 5), (13010,), (3253,))

In [11]:
# Standardize the features
scalerX = MinMaxScaler().fit(X_train)
X_train = pd.DataFrame(scalerX.transform(X_train), columns = ES_EZ_call_X.columns.values) 
X_test = pd.DataFrame(scalerX.transform(X_test), columns = ES_EZ_call_X.columns.values) 

scalery = MinMaxScaler().fit(y_train.values.reshape(-1, 1))
y_train = scalery.transform(y_train.values.reshape(-1, 1))
y_test = scalery.transform(y_test.values.reshape(-1, 1))

In [12]:
learning_rate = [1e-2, 1e-3, 1e-4]
History = []
times = []
earlyStop = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 3)
for lr in learning_rate:
    model = models.Sequential()
    # Input layer
    model.add(layers.Dense(units = 5))
    
    numLayers = 4
    for i in range(numLayers):
        model.add(layers.Dense(units = 100))
        model.add(BatchNormalization())
        model.add(layers.Activation(activations.elu))
        model.add(layers.Dropout(0.25))

    # Output layer
    model.add(layers.Dense(units = 1, activation = "exponential"))

    model.compile(
        optimizer = keras.optimizers.Adam(learning_rate = lr), loss = "mean_squared_error"
    )
    
    time_callback = TimeHistory()

    history = model.fit(X_train, 
        y_train, 
        epochs = 100, 
        verbose = 0,
        batch_size = 32, 
        callbacks = [earlyStop, time_callback],
        validation_split = 0.2)
    BSNN_train_pred = model.predict(X_train)
    BSNN_test_pred = model.predict(X_test)
    print('When the learning rate is', lr)
    print("The training R squared is", 
          np.round(r2_score(y_train, BSNN_train_pred), 4))
    print("The testing R squared is", 
          np.round(r2_score(y_test, BSNN_test_pred), 4))
    print('-------------------------------------------------------------------')
    History.append(history)
    times.append(time_callback.times)
    path = "/Users/gaojinglun/Desktop/RSG/ANN_put_TrainOnESEZ_withLR" + str(lr)
    model.save(path)

When the learning rate is 0.01
The training R squared is 0.9596
The testing R squared is 0.9588
-------------------------------------------------------------------
INFO:tensorflow:Assets written to: /Users/gaojinglun/Desktop/RSG/ANN_put_TrainOnESEZ_withLR0.01/assets
When the learning rate is 0.001
The training R squared is 0.9753
The testing R squared is 0.9734
-------------------------------------------------------------------
INFO:tensorflow:Assets written to: /Users/gaojinglun/Desktop/RSG/ANN_put_TrainOnESEZ_withLR0.001/assets
When the learning rate is 0.0001
The training R squared is 0.8577
The testing R squared is 0.8554
-------------------------------------------------------------------
INFO:tensorflow:Assets written to: /Users/gaojinglun/Desktop/RSG/ANN_put_TrainOnESEZ_withLR0.0001/assets
